In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [19]:
train = pd.read_csv("/kaggle/input/samplednoinfnonnorm/RandomSamplingSMOTE_train_Sampled_NOINF_nonnorm.csv")
test = pd.read_csv("/kaggle/input/noinfnonnormtestt/X_test.csv")

In [20]:
X_fe_train = train.copy()
X_fe_train.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5
0,6,117986095,11,9,875,355,517,0,79.545455,154.043087,...,5.862518e+07,374022.010588,58889657,58360710,1.0,0.0,0.0,0.0,0.0,0.0
1,6,86160336,2,0,0,0,0,0,0.000000,0.000000,...,8.616034e+07,0.000000,86160336,86160336,1.0,0.0,0.0,0.0,0.0,0.0
2,6,3769550,14,8,1439,1731,725,0,102.785714,191.197111,...,0.000000e+00,0.000000,0,0,1.0,0.0,0.0,0.0,0.0,0.0
3,6,61337501,21,19,2727,9591,724,0,129.857143,242.631054,...,9.990232e+06,32318.260496,10003474,9924263,1.0,0.0,0.0,0.0,0.0,0.0
4,17,2573,1,1,30,207,30,30,30.000000,0.000000,...,0.000000e+00,0.000000,0,0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X_fe_test = test.copy()
X_fe_test.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5
0,6,2471708,8,8,1148,1581,677,0,143.500000,228.129662,...,0.0,0.0,0,0,1,0,0,0,0,0
1,6,816,3,0,77,0,46,0,25.666667,23.459184,...,0.0,0.0,0,0,1,0,0,0,0,0
2,6,2495557,9,8,1789,1581,661,0,198.777778,270.943619,...,0.0,0.0,0,0,1,0,0,0,0,0
3,6,116,2,1,38,0,38,0,19.000000,26.870058,...,0.0,0.0,0,0,1,0,0,0,0,0
4,6,3958451,14,8,1441,1731,725,0,102.928571,191.148941,...,0.0,0.0,0,0,1,0,0,0,0,0


In [22]:
last_6_columns = X_fe_train.iloc[:, -6:]
Y_fe_train = pd.DataFrame(last_6_columns)
Y_fe_train.head()

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
Y_fe_train.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
dtype: object

In [24]:
last_6_columns_test = X_fe_test.iloc[:, -6:]
Y_fe_test = pd.DataFrame(last_6_columns_test)
Y_fe_test.head()

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [25]:
X_fe_train = X_fe_train.drop(X_fe_train.columns[-6:], axis=1)
X_fe_train.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,6,117986095,11,9,875,355,517,0,79.545455,154.043087,...,6,20,289957.000000,189894.940301,424233,155681,5.862518e+07,374022.010588,58889657,58360710
1,6,86160336,2,0,0,0,0,0,0.000000,0.000000,...,0,20,0.000000,0.000000,0,0,8.616034e+07,0.000000,86160336,86160336
2,6,3769550,14,8,1439,1731,725,0,102.785714,191.197111,...,7,20,0.000000,0.000000,0,0,0.000000e+00,0.000000,0,0
3,6,61337501,21,19,2727,9591,724,0,129.857143,242.631054,...,12,20,195588.333333,449991.678643,1114130,11845,9.990232e+06,32318.260496,10003474,9924263
4,17,2573,1,1,30,207,30,30,30.000000,0.000000,...,0,8,0.000000,0.000000,0,0,0.000000e+00,0.000000,0,0


In [26]:
X_fe_test = X_fe_test.drop(X_fe_test.columns[-6:], axis=1)
X_fe_test.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,6,2471708,8,8,1148,1581,677,0,143.500000,228.129662,...,5,20,0.0,0.0,0,0,0.0,0.0,0,0
1,6,816,3,0,77,0,46,0,25.666667,23.459184,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
2,6,2495557,9,8,1789,1581,661,0,198.777778,270.943619,...,6,20,0.0,0.0,0,0,0.0,0.0,0,0
3,6,116,2,1,38,0,38,0,19.000000,26.870058,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
4,6,3958451,14,8,1441,1731,725,0,102.928571,191.148941,...,7,20,0.0,0.0,0,0,0.0,0.0,0,0


In [27]:
X_fe_test.dtypes

Protocol             int64
Flow Duration        int64
Tot Fwd Pkts         int64
Tot Bwd Pkts         int64
TotLen Fwd Pkts      int64
                    ...   
Active Min           int64
Idle Mean          float64
Idle Std           float64
Idle Max             int64
Idle Min             int64
Length: 69, dtype: object

# Using RFE and RF
* Decision Tree
* AdaBoost
* RF 
* XGBoost 
* CatBoost 

[www.evidentlyai.com/classification-metrics/multi-class-metrics](https://)

In [28]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_fe_train, Y_fe_train)
sel.get_support()

X_fe_train.columns

features = X_fe_train.columns[sel.get_support()]
print(features)
print(len(features))

np.mean(sel.estimator_.feature_importances_)

X_train_rfe = sel.transform(X_fe_train)
X_test_rfe = sel.transform(X_fe_test)

Index(['Flow Duration', 'Tot Fwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Max', 'ECE Flag Cnt', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts',
       'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min'],
      dtype='object')
32


In [29]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
training_times = []

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    start_time = time.time()
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    end_time = time.time()
    training_time = end_time - start_time
    training_times.append(training_time)
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [34]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.758359,0.144558,0.167186,0.343482
Random Forest,0.998823,0.764020,0.727824,0.879506
XGBoost,0.999032,0.761401,0.716411,0.894504
CatBoost,0.998085,0.693676,0.624964,0.920844
Decision Tree,0.999439,0.760883,0.711050,0.863638


In [35]:
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.9994392623391369 achieved by: Decision Tree
Max F1 score:  0.7640200559479257  achieved by  Random Forest
Max Precision:  0.727824103647169  achieved by  Random Forest
Max Recall:  0.9208442406933225  achieved by  CatBoost


In [ ]:
ax = acc_all.plot.bar(figsize=(15, 8))
for p in ax.patches:
    ax.annotate(str(p.get_height().round(3)), (p.get_x()*0.985, p.get_height()*1.002))
plt.ylim((0.10, 1.00))
plt.xticks(rotation=90)
plt.title("All Classifier Accuracies")
plt.grid()
plt.show()

In [36]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

max_accuracy_per_class = {}
for accuracy_dict in acc_per_class_rfe:
    for c, accuracy in accuracy_dict.items():
        if c not in max_accuracy_per_class or accuracy > max_accuracy_per_class[c]:
            max_accuracy_per_class[c] = accuracy

print(max_accuracy_per_class)

                                                                                                                       Feature-selected dataset
AdaBoost       {0: 0.7932433781161985, 1: 0.25225225225225223, 2: 0.9565217391304348, 3: 0.0, 4: 0.058823529411764705, 5: 5.31632110579479e-05}
Random Forest        {0: 0.998851014004484, 1: 0.8378378378378378, 2: 0.9347826086956522, 3: 0.8, 4: 0.7058823529411765, 5: 0.9996810207336523}
XGBoost             {0: 0.9990530602706447, 1: 0.8468468468468469, 2: 0.9565217391304348, 3: 0.8, 4: 0.7647058823529411, 5: 0.9998936735778841}
CatBoost            {0: 0.9980623032787483, 1: 0.8648648648648649, 2: 0.9565217391304348, 3: 1.0, 4: 0.7058823529411765, 5: 0.9997341839447103}
Decision Tree         {0: 0.99949610148078, 1: 0.8378378378378378, 2: 0.9565217391304348, 3: 0.8, 4: 0.5882352941176471, 5: 0.9997341839447103}
{0: 0.99949610148078, 1: 0.8648648648648649, 2: 0.9565217391304348, 3: 1.0, 4: 0.7647058823529411, 5: 0.9998936735778841}


# Using RFE and Variance Based Selection (not possible with KNN)

In [37]:
from sklearn.feature_selection import VarianceThreshold
var_threshold = 0.1  
selector = VarianceThreshold(threshold=var_threshold)
selector.fit(X_fe_train)
selected_features = X_fe_train.columns[selector.get_support()]
print("Selected features:", selected_features)
print("Number of selected features:", len(selected_features))
X_train_rfe = selector.transform(X_fe_train)
X_test_rfe = selector.transform(X_fe_test)

Selected features: Index(['Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts'

In [38]:
Y_fe_train_1d = np.argmax(Y_fe_train.values, axis=1)
training_times = []

random_state = 42
clf_rfe_keys = ["AdaBoost"]
clf_rfe_values = [AdaBoostClassifier(random_state=random_state)]

unique_classes = np.unique(Y_fe_train_1d)
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
Y_fe_train_1d_mapped = np.vectorize(class_mapping.get)(Y_fe_train_1d)

clf_rfe_keys += ["Random Forest", "XGBoost", "CatBoost","Decision Tree",]
clf_rfe_values += [RandomForestClassifier(random_state=random_state), 
                    XGBClassifier(random_state=random_state), 
                    CatBoostClassifier(random_state=random_state, verbose=0),DecisionTreeClassifier(random_state=random_state)]

clfs_rfe = dict(zip(clf_rfe_keys, clf_rfe_values))

print("Model training using feature-selected data: started!")
for clf_rfe_name, clf_rfe in clfs_rfe.items():
    start_time = time.time()
    if clf_rfe_name == 'XGBoost':
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d_mapped)
    else:
        clf_rfe.fit(X_train_rfe, Y_fe_train_1d)
    clfs_rfe[clf_rfe_name] = clf_rfe
    end_time = time.time()
    training_time = end_time - start_time
    training_times.append(training_time)
    print(clf_rfe_name, "training: done!", )
print("Model training using feature-selected data: done!")

Model training using feature-selected data: started!
AdaBoost training: done!
Random Forest training: done!
XGBoost training: done!
CatBoost training: done!
Decision Tree training: done!
Model training using feature-selected data: done!


In [40]:
Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)

acc_rfe = []
f1_rfe = []
precision_rfe = []
recall_rfe = []

max_f1 = {"score": 0, "model": ""}
max_precision = {"score": 0, "model": ""}
max_recall = {"score": 0, "model": ""}

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    
    f1 = f1_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    precision = precision_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    recall = recall_score(Y_fe_test_1d, y_rfe_pred, average='macro')
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    
    f1_rfe.append(f1)
    precision_rfe.append(precision)
    recall_rfe.append(recall)
    
    # Update maximum metrics and corresponding model names
    if f1 > max_f1["score"]:
        max_f1 = {"score": f1, "model": clf_rfe_name}
    if precision > max_precision["score"]:
        max_precision = {"score": precision, "model": clf_rfe_name}
    if recall > max_recall["score"]:
        max_recall = {"score": recall, "model": clf_rfe_name}    
    
    
acc_all = pd.DataFrame({"Feature-selected dataset": acc_rfe},index=clf_rfe_keys)

metrics_all = pd.DataFrame({
    "Feature-selected dataset - Accuracy": acc_rfe,
    "Feature-selected dataset - F1 Score": f1_rfe,
    "Feature-selected dataset - Precision": precision_rfe,
    "Feature-selected dataset - Recall": recall_rfe,
}, index=clf_rfe_keys)

metrics_all

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Feature-selected dataset - Accuracy,Feature-selected dataset - F1 Score,Feature-selected dataset - Precision,Feature-selected dataset - Recall
AdaBoost,0.696471,0.137376,0.157045,0.320590
Random Forest,0.999311,0.784975,0.748505,0.868471
XGBoost,0.999109,0.758081,0.718807,0.881099
CatBoost,0.998862,0.708256,0.635869,0.916476
Decision Tree,0.999504,0.770927,0.724854,0.863037


In [41]:
# Print maximum metrics and corresponding model names
max_acc_rfe = acc_all["Feature-selected dataset"].max()
best_model_rfe = acc_all[acc_all["Feature-selected dataset"] == max_acc_rfe].index[0]
print("Max Accuracy:", max_acc_rfe, "achieved by:", best_model_rfe)
print("Max F1 score: ", max_f1["score"], " achieved by ", max_f1["model"])
print("Max Precision: ", max_precision["score"], " achieved by ", max_precision["model"])
print("Max Recall: ", max_recall["score"], " achieved by ", max_recall["model"])

Max Accuracy: 0.9995044102831376 achieved by: Decision Tree
Max F1 score:  0.7849747767065921  achieved by  Random Forest
Max Precision:  0.7485050484898673  achieved by  Random Forest
Max Recall:  0.9164764622765612  achieved by  CatBoost


In [43]:
def calculate_accuracy_per_class(y_true, y_pred):
    classes = set(y_true)
    accuracy_per_class = {}
    for c in classes:
        samples = [y for y, pred in zip(y_true, y_pred) if y == c]
        correctly_classified = [y for y, pred in zip(y_true, y_pred) if y == c and y == pred]
        accuracy_per_class[c] = len(correctly_classified) / len(samples) if samples else 0
    return accuracy_per_class

Y_fe_test_1d = np.argmax(Y_fe_test.values, axis=1)
acc_rfe = []
acc_per_class_rfe = []

for clf_rfe_name, clf_rfe in clfs_rfe.items():
    y_rfe_pred = clf_rfe.predict(X_test_rfe)
    acc_rfe.append(accuracy_score(Y_fe_test_1d, y_rfe_pred))
    accuracy_per_class = calculate_accuracy_per_class(Y_fe_test_1d, y_rfe_pred)
    acc_per_class_rfe.append(accuracy_per_class)

acc_all = pd.DataFrame({"Feature-selected dataset": acc_per_class_rfe},index=clf_rfe_keys)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(acc_all)

max_accuracy_per_class = {}
for accuracy_dict in acc_per_class_rfe:
    for c, accuracy in accuracy_dict.items():
        if c not in max_accuracy_per_class or accuracy > max_accuracy_per_class[c]:
            max_accuracy_per_class[c] = accuracy

print(max_accuracy_per_class)

                                                                                                                  Feature-selected dataset
AdaBoost                       {0: 0.72853258422043, 1: 0.0990990990990991, 2: 0.9782608695652174, 3: 0.0, 4: 0.11764705882352941, 5: 0.0}
Random Forest  {0: 0.9993743868626109, 1: 0.7927927927927928, 2: 0.9130434782608695, 3: 0.8, 4: 0.7058823529411765, 5: 0.9997341839447103}
XGBoost         {0: 0.9991358262109996, 1: 0.8468468468468469, 2: 0.9347826086956522, 3: 0.8, 4: 0.7058823529411765, 5: 0.999946836788942}
CatBoost        {0: 0.998882659805208, 1: 0.8378378378378378, 2: 0.9565217391304348, 3: 1.0, 4: 0.7058823529411765, 5: 0.9997341839447103}
Decision Tree   {0: 0.999559393082228, 1: 0.8558558558558559, 2: 0.9347826086956522, 3: 0.8, 4: 0.5882352941176471, 5: 0.9997873471557682}
{0: 0.999559393082228, 1: 0.8558558558558559, 2: 0.9782608695652174, 3: 1.0, 4: 0.7058823529411765, 5: 0.999946836788942}
